# Crime Classification – CS 9548 Project
**Goal:** Exploring Machine Learning Techniques for Image Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download and Extract Dataset

In [ ]:
!pip install kagglehub

import kagglehub
path = kagglehub.dataset_download("odins0n/ucf-crime-dataset")

print("Path to dataset files:", path)

100%|██████████| 11.0G/11.0G [08:24<00:00, 23.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/odins0n/ucf-crime-dataset/versions/1


## Create DataFrame of Images

In [ ]:
import pathlib
import pandas as pd

# Get directories of train and test datasets
data_dir = pathlib.Path(path)
train_dir = data_dir / "Train"
test_dir = data_dir / "Test"

print("Data dir:", data_dir)
print(train_dir)
print(test_dir)

# Function to build dataframe
def build_image_df(root_dir):
    root_dir = pathlib.Path(root_dir)
    image_paths = list(root_dir.glob("*/*.png"))

    rows = []

    for p in image_paths:
        label = p.parent.name
        rows.append({"image": str(p), "label": label})

    return pd.DataFrame(rows)


# Build train and test dataframes
train_df = build_image_df(train_dir)
test_df = build_image_df(test_dir)

print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

Data dir: /root/.cache/kagglehub/datasets/odins0n/ucf-crime-dataset/versions/1
/root/.cache/kagglehub/datasets/odins0n/ucf-crime-dataset/versions/1/Train
/root/.cache/kagglehub/datasets/odins0n/ucf-crime-dataset/versions/1/Test
label
NormalVideos     947768
Stealing          44802
Robbery           41493
Burglary          39504
Arrest            26397
Shoplifting       24835
Fighting          24684
Arson             24421
RoadAccidents     23486
Abuse             19076
Explosion         18753
Vandalism         13626
Assault           10360
Shooting           7140
Name: count, dtype: int64
label
NormalVideos     64952
Burglary          7657
Shooting          7630
Shoplifting       7623
Explosion         6510
Arrest            3365
Arson             2793
RoadAccidents     2663
Assault           2657
Stealing          1984
Fighting          1231
Vandalism         1111
Robbery            835
Abuse              297
Name: count, dtype: int64


## Encode Label as Integer

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create and fit label encoder on training labels
le = LabelEncoder()
train_df['label_idx'] = le.fit_transform(train_df['label'])

# Apply the same encoding to test labels
test_df['label_idx']  = le.transform(test_df['label'])

# Number of classes and mapping
num_classes = len(le.classes_)
print("Number of classes:", num_classes)
print("Class name vs label_idx:")
for index, cls in enumerate(le.classes_):
    print(index, cls)

Number of classes: 14
Class name vs label_idx:
0 Abuse
1 Arrest
2 Arson
3 Assault
4 Burglary
5 Explosion
6 Fighting
7 NormalVideos
8 RoadAccidents
9 Robbery
10 Shooting
11 Shoplifting
12 Stealing
13 Vandalism


We can see that the dataset has a large class imbalance. To mitigate this, we can do some under/oversampling, as well as data augmentation.

## Add Video Column ID

In [ ]:
def extract_video_id(path_str):
    stem = pathlib.Path(path_str).stem

    # split off the last chunk
    video_id = stem.rsplit("_", 1)[0]

    return video_id

train_df['video_id'] = train_df['image'].apply(extract_video_id)
test_df['video_id']  = test_df['image'].apply(extract_video_id)

## Split Train Into Train and Val

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

unique_videos = train_df['video_id'].unique()

video_labels = (
    train_df.groupby('video_id')['label_idx'].agg(lambda x: np.bincount(x).argmax()).reindex(unique_videos)
)

# 80% train 20% val
train_vids, val_vids = train_test_split(
    unique_videos,
    test_size=0.20,
    random_state=42,
    stratify=video_labels
)

train_df_raw = train_df[train_df['video_id'].isin(train_vids)].reset_index(drop=True)
val_df = train_df[train_df['video_id'].isin(val_vids)].reset_index(drop=True)

print("\nNum train videos:", len(train_vids))
print("Num val videos:", len(val_vids))
print("Train frames:", len(train_df_raw))
print("Val frames:", len(val_df))
print("Test frames:", len(test_df))


Num train videos: 1288
Num val videos: 322
Train frames: 1047139
Val frames: 219206
Test frames: 111308


# No Over/undersampling

In [ ]:
TARGET = 10000

def balance_dataset(df):
    balanced_parts = []

    for cls, group in df.groupby("label"):
        count = len(group)

        if count > TARGET:
            # Only undersample big classes
            group = group.sample(
                n=TARGET,
                replace=False,
                random_state=42
            )
        # if count <= TARGET: keep as-is

        balanced_parts.append(group)

    return pd.concat(balanced_parts).sample(frac=1, random_state=42).reset_index(drop=True)

train_df_used = balance_dataset(train_df_raw)

In [ ]:
print(train_df_used['label'].value_counts())

label
Explosion        10000
Abuse            10000
Robbery          10000
Burglary         10000
Arrest           10000
Fighting         10000
RoadAccidents    10000
Shoplifting      10000
Arson            10000
Stealing         10000
NormalVideos     10000
Vandalism         9669
Assault           8955
Shooting          5807
Name: count, dtype: int64


## Set up PyTorch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Set constants for PyTorch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

## Data Augmentation

In [ ]:
# Data augmentation to training dataset
train_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.9, 1.1)),
    transforms.ColorJitter(contrast=0.1),
    transforms.ToTensor()
])

# Simple transforms to val/test dataset
eval_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

## Create Pipeline For PyTorch

In [ ]:
class CrimeDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = int(row['label_idx'])

        # Load the image
        image = Image.open(img_path).convert("RGB")

        # Transform if needed (might not need for val/test)
        if self.transform:
            image = self.transform(image)

        return image, label

## Create DataLoaders

In [ ]:
# First convert to CrimeDataset classes
train_dataset = CrimeDataset(train_df_used, train_transform)
val_dataset = CrimeDataset(val_df, eval_transform)
test_dataset = CrimeDataset(test_df, eval_transform)

# Transform to DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
images, labels = next(iter(train_loader))
print("\nSample batch shapes -> images:", images.shape, "labels:", labels.shape)


Sample batch shapes -> images: torch.Size([32, 3, 64, 64]) labels: torch.Size([32])


## Create ResNet18 Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Class-weighted loss
criterion = nn.CrossEntropyLoss()

# Add L2 regularization via weight_decay
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


## Training and Testing

In [23]:
import torch.optim as optim

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [24]:
def test(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.inference_mode():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [25]:
EPOCHS = 10

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = test(model, val_loader, criterion, device)

    print(
        f"Epoch {epoch}/{EPOCHS} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )

Epoch 1/10 | Train Loss: 0.4333, Train Acc: 0.8702 | Val Loss: 2.5429, Val Acc: 0.4623
Epoch 2/10 | Train Loss: 0.1602, Train Acc: 0.9531 | Val Loss: 2.2687, Val Acc: 0.4598
Epoch 3/10 | Train Loss: 0.1227, Train Acc: 0.9635 | Val Loss: 2.5113, Val Acc: 0.3784
Epoch 4/10 | Train Loss: 0.1073, Train Acc: 0.9683 | Val Loss: 2.3734, Val Acc: 0.4478
Epoch 5/10 | Train Loss: 0.0947, Train Acc: 0.9721 | Val Loss: 2.5753, Val Acc: 0.4946
Epoch 6/10 | Train Loss: 0.0895, Train Acc: 0.9729 | Val Loss: 2.6204, Val Acc: 0.4596
Epoch 7/10 | Train Loss: 0.0827, Train Acc: 0.9747 | Val Loss: 2.7524, Val Acc: 0.4211
Epoch 8/10 | Train Loss: 0.0788, Train Acc: 0.9763 | Val Loss: 2.5320, Val Acc: 0.4123
Epoch 9/10 | Train Loss: 0.0779, Train Acc: 0.9766 | Val Loss: 2.1406, Val Acc: 0.5864
Epoch 10/10 | Train Loss: 0.0739, Train Acc: 0.9779 | Val Loss: 2.4603, Val Acc: 0.5245


In [26]:
test_loss, test_acc = test(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


Test Loss: 3.3308, Test Acc: 0.3481
